# **Audio Clip Feature Extraction**

This script processes a directory of sliced audio clips and converts them into a model-ready dataset.

**Workflow:**

* Iterates through organized language folders (igbo, hausa, yoruba).

* Converts each .wav clip into a Mel spectrogram using librosa.

* Resizes every spectrogram to a uniform shape of (128, 256) by trimming or padding.

* Saves each spectrogram as an individual .npy file in a features/ directory to conserve RAM.

* Generates a metadata.csv file that maps each feature file to its correct numerical label.

**Output:**

* A features/ folder containing all .npy spectrogram files.

* A metadata.csv file linking each feature to its label.

In [ ]:
.from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import librosa
import numpy as np
from pathlib import Path
import csv
from pathlib import Path

def extract_features(source_dir, features_dir, label_map, target_shape=(128, 256)):
  """
  Processes audio clips from a source directory, extracts Mel spectrograms, saves them
  as individual .npy files, and creates a metadata.csv file.

  Args:
    source_dir (Path): The root directory of the segmented audio clips.
    features_dir (Path): The destination directory to save feature files.
    label_map (dict): A dictionary mapping language folder names to integer labels.
    target_shape (tuple): The desired (height, width) for the spectrograms.
  """

  # Create the destination directory if it doesn't exist
  features_dir.mkdir(parents=True, exist_ok=True)

  metadata = []  # This will store ['path/to/feature.npy', label]

  print("Starting feature extraction...")
  for folder in source_dir.iterdir():
    if folder.is_dir():
      language_name = folder.name
      label = label_map.get(language_name)

      if label is None:
        continue

      print(f"\nProcessing language: {language_name} (Label: {label})")

      for sub_folder in folder.iterdir():
        if sub_folder.is_dir():
          for audio_file in sub_folder.glob('*.wav'):
            try:
              y, sr = librosa.load(audio_file, sr=16000)

              if len(y) < 400:
                continue

              mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=target_shape[0])
              log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

              current_width = log_mel_spectrogram.shape[1]
              target_width = target_shape[1]

              if current_width > target_width:
                uniform_spectrogram = log_mel_spectrogram[:, :target_width]
              else:
                padding = np.zeros((target_shape[0], target_width - current_width))
                uniform_spectrogram = np.hstack((log_mel_spectrogram, padding))

              # Create a unique filename for the spectrogram
              output_filename = f"{sub_folder.name}_{audio_file.stem}.npy"
              output_path = features_dir / output_filename

              # Save the NumPy array to the new file
              np.save(output_path, uniform_spectrogram)

              # Add the file path and label to our metadata list
              metadata.append([str(output_path), label])

            except Exception as e:
              print(f"  Error processing {audio_file}: {e}")

  # Save the metadata to a CSV file
  metadata_path = features_dir.parent / "metadata.csv"
  try:
    with open(metadata_path, 'w', newline='', encoding='utf-8') as f:
      writer = csv.writer(f)
      writer.writerow(['feature_path', 'label'])
      writer.writerows(metadata)
    print(f"\nMetadata successfully saved to: {metadata_path}")
  except Exception as e:
    print(f"Error saving metadata.csv: {e}")

  print("\n-------------------------------------")
  print(f"Feature extraction complete! Total features saved: {len(metadata)}")
  return metadata_path


# 1. Define the necessary parameters
source_directory = Path("/content/drive/MyDrive/Naija-Language-Accent_ID/Data/processed_audio")
features_directory = Path("/content/drive/MyDrive/Naija-Language-Accent_ID/Data/features")
language_labels = {'spoken_igbo': 0, 'spoken_hausa': 1, 'spoken_yoruba': 2}

# 2. Call the function to run the entire process
metadata_file_path = extract_features(source_dir=source_directory,
                                      features_dir=features_directory,
                                      label_map=language_labels)

print(f"\nProcess finished. Metadata is located at: {metadata_file_path}")